# Example 0

This notebook generates dark neutrino events, loads them, and describes the output formats. 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import pandas as pd

import DarkNews as dn
from DarkNews import const

## Generating events
Let us start by generating a few dark neutrino events for two cases of interest.

We do this in two different ways. 

### Command line functionality
It is possible to run the generator through the script `dn_gen`, passing the parameters as options.
The created dataset can be found in the `data/` directory tree, which is created in the same folder the script is run.

Let's try to run few points.

In [3]:
cmd_string = "dn_gen --mzprime=1.25 --m4=0.140 --neval=1000 --D_or_M=dirac --log=INFO"
const.subprocess_cmd(cmd_string)

dn_gen --mzprime=1.25 --m4=0.140 --neval=1000 --D_or_M=dirac --log=INFO


#########################################################
#   ______           _        _   _                     #
#   |  _  \         | |      | \ | |                    #
#   | | | |__ _ _ __| | __   |  \| | _____      _____   #
#   | | | / _  | ___| |/ /   | .   |/ _ \ \ /\ / / __|  #
#   | |/ / (_| | |  |   <    | |\  |  __/\ V  V /\__ \  #
#   |___/ \__,_|_|  |_|\_\   \_| \_/\___| \_/\_/ |___/  #
#                                                       #
#########################################################
        
Theory model used: 3+1 dirac HNL model


Generating helicity conserving upscattering events for:
	nu(mu) + C12 -> N4 +  C12 -> nu(mu) + e+ + e- + C12

decaying N4 using off-shell mediator.
Generating helicity flipping upscattering events for:
	nu(mu) + C12 -> N4 +  C12 -> nu(mu) + e+ + e- + C12

decaying N4 using off-shell mediator.
Generating helicity conserving upscattering events for:
	nu(

Load the dataset.

In [4]:
df_d = pd.read_pickle('./data/miniboone_fhc/3plus1/m4_0.14_mzprime_1.25_dirac/pandas_df.pckl')
df_d

P_projectile                     P_decay_N_parent                      \
                t    x    y         z                t         x         y   
0        0.421838  0.0  0.0  0.421838         0.421581  0.037286 -0.058601   
1        0.607662  0.0  0.0  0.607662         0.607317 -0.066828  0.052254   
2        0.518768  0.0  0.0  0.518768         0.518538  0.013953 -0.066121   
3        0.581459  0.0  0.0  0.581459         0.580975  0.030147 -0.095975   
4        0.584691  0.0  0.0  0.584691         0.584612  0.031907 -0.019789   
...           ...  ...  ...       ...              ...       ...       ...   
5356     1.111706  0.0  0.0  1.111706         0.973739 -0.100894  0.445498   
5357     1.297461  0.0  0.0  1.297461         0.736012  0.110894 -0.637062   
5358     1.218458  0.0  0.0  1.218458         1.055309 -0.090798  0.486236   
5359     1.179622  0.0  0.0  1.179622         0.542911  0.293014 -0.434168   
5360     1.693356  0.0  0.0  1.693356         1.488808  0.179357  0.537354   

                 P_target       ... decay_displacement      w_decay_rate_0  \
             z          t    x  ...                  y    z                  
0     0.391544  11.174864 -0.0  ...               -0.0  0.0   3.601060e-23   
1     0.584839  11.174864 -0.0  ...                0.0  0.0   2.159990e-23   
2     0.494687  11.174864 -0.0  ...               -0.0  0.0   2.562573e-23   
3     0.554808  11.174864 -0.0  ...               -0.0  0.0   4.953109e-24   
4     0.566359  11.174864 -0.0  ...               -0.0  0.0   7.837724e-23   
...        ...        ...  ...  ...                ...  ...            ...   
5356  0.848481   0.938272 -0.0  ...                0.0  0.0   1.929055e-23   
5357  0.322442   0.938272 -0.0  ...               -0.0  0.0   4.537432e-23   
5358  0.921633   0.938272 -0.0  ...                0.0  0.0   2.170112e-23   
5359  0.028163   0.938272 -0.0  ...               -0.0  0.0   4.740506e-23   
5360  1.369683   0.938272 -0.0  ...                0.0  0.0   1.205715e-22   

     w_event_rate w_flux_avg_xsec        target target_pdgid  \
                                                               
0        0.000202        0.000040           C12   1000060120   
1        0.000192        0.000042           C12   1000060120   
2        0.000160        0.000040           C12   1000060120   
3        0.000051        0.000038           C12   1000060120   
4        0.000214        0.000033           C12   1000060120   
...           ...             ...           ...          ...   
5356     0.000068        0.000043  proton_in_H1         2212   
5357     0.000030        0.000012  proton_in_H1         2212   
5358     0.000063        0.000044  proton_in_H1         2212   
5359     0.000029        0.000013  proton_in_H1         2212   
5360     0.000095        0.000094  proton_in_H1         2212   

     scattering_regime    helicity  \
                                     
0             coherent  conserving   
1             coherent  conserving   
2             coherent  conserving   
3             coherent  conserving   
4             coherent  conserving   
...                ...         ...   
5356              p-el    flipping   
5357              p-el    flipping   
5358              p-el    flipping   
5359              p-el    flipping   
5360              p-el    flipping   

                                     underlying_process  
                                                         
0     nu(mu) + C12 -> N4 +  C12 -> nu(mu) + e+ + e- ...  
1     nu(mu) + C12 -> N4 +  C12 -> nu(mu) + e+ + e- ...  
2     nu(mu) + C12 -> N4 +  C12 -> nu(mu) + e+ + e- ...  
3     nu(mu) + C12 -> N4 +  C12 -> nu(mu) + e+ + e- ...  
4     nu(mu) + C12 -> N4 +  C12 -> nu(mu) + e+ + e- ...  
...                                                 ...  
5356  nu(mu) + proton_in_H1 -> N4 +  proton_in_H1 ->...  
5357  nu(mu) + proton_in_H1 -> N4 +  proton_in_H1 ->...  
5358  nu(mu) + proton_in_H1 -> N4 +  proton_in_H1 ->...  
5359  nu(

In [5]:
cmd_string = "dn_gen --umu5=1e-3 --ud5=35 --chi=0.0031 --gD=2 --mzprime=1.25 --m4=0.080 --m5=0.140 --neval=1000 --D_or_M=majorana --log=INFO"
const.subprocess_cmd(cmd_string)

dn_gen --umu5=1e-3 --ud5=35 --chi=0.0031 --gD=2 --mzprime=1.25 --m4=0.080 --m5=0.140 --neval=1000 --D_or_M=majorana --log=INFO


#########################################################
#   ______           _        _   _                     #
#   |  _  \         | |      | \ | |                    #
#   | | | |__ _ _ __| | __   |  \| | _____      _____   #
#   | | | / _  | ___| |/ /   | .   |/ _ \ \ /\ / / __|  #
#   | |/ / (_| | |  |   <    | |\  |  __/\ V  V /\__ \  #
#   |___/ \__,_|_|  |_|\_\   \_| \_/\___| \_/\_/ |___/  #
#                                                       #
#########################################################
        
Theory model used: 3+2 majorana HNL model


Generating helicity conserving upscattering events for:
	nu(mu) + C12 -> N5 +  C12 -> N4 + e+ + e- + C12

decaying N5 using off-shell mediator.
Generating helicity flipping upscattering events for:
	nu(mu) + C12 -> N5 +  C12 -> N4 + e+ + e- + C12

decaying N5 using off-shell mediator.
Generating

In [15]:
df_d = pd.read_pickle('./data/miniboone_fhc/3plus2/m5_0.14_m4_0.08_mzprime_1.25_majorana/pandas_df.pckl')
df_d

P_projectile                     P_decay_N_parent                      \
                t    x    y         z                t         x         y   
0        0.600389  0.0  0.0  0.600389         0.599990  0.009691  0.090732   
1        0.298606  0.0  0.0  0.298606         0.297957  0.006894 -0.105472   
2        0.203648  0.0  0.0  0.203648         0.202908 -0.073001  0.056656   
3        0.584652  0.0  0.0  0.584652         0.584382 -0.056977 -0.047872   
4        0.581939  0.0  0.0  0.581939         0.581735  0.008025 -0.063713   
...           ...  ...  ...       ...              ...       ...       ...   
5365     1.480923  0.0  0.0  1.480923         0.626242  0.450097 -0.404832   
5366     1.284102  0.0  0.0  1.284102         0.912066  0.616349 -0.179454   
5367     1.110781  0.0  0.0  1.110781         0.643833  0.380303 -0.438646   
5368     8.089877  0.0  0.0  8.089877         6.788148  0.648404  1.266077   
5369     7.903061  0.0  0.0  7.903061         6.932040 -0.302565 -1.220962   

                 P_target       ... decay_displacement      w_decay_rate_0  \
             z          t    x  ...                  y    z                  
0     0.576248  11.174864 -0.0  ...                0.0  0.0   3.976789e-16   
1     0.240845  11.174864 -0.0  ...               -0.0  0.0   5.750206e-16   
2     0.114160  11.174864 -0.0  ...                0.0  0.0   9.177723e-16   
3     0.562463  11.174864 -0.0  ...               -0.0  0.0   5.631801e-16   
4     0.560974  11.174864 -0.0  ...               -0.0  0.0   1.015882e-15   
...        ...        ...  ...  ...                ...  ...            ...   
5365  0.078123   0.938272 -0.0  ...               -0.0  0.0   2.136635e-15   
5366  0.632593   0.938272 -0.0  ...               -0.0  0.0   7.227626e-16   
5367  0.240581   0.938272 -0.0  ...               -0.0  0.0   7.039601e-16   
5368  6.635961   0.938272 -0.0  ...                0.0  0.0   1.707376e-13   
5369  6.815518   0.938272 -0.0  ...               -0.0  0.0   1.450069e-13   

     w_event_rate w_flux_avg_xsec        target target_pdgid  \
                                                               
0       19.731114       13.008118           C12   1000060120   
1       16.920468       13.808261           C12   1000060120   
2       12.664077        9.128393           C12   1000060120   
3       21.887229       15.000501           C12   1000060120   
4       32.298143       25.976216           C12   1000060120   
...           ...             ...           ...          ...   
5365     3.870794        1.895771  proton_in_H1         2212   
5366    10.349804        5.019607  proton_in_H1         2212   
5367     9.166310        5.195656  proton_in_H1         2212   
5368     0.000000        0.000000  proton_in_H1         2212   
5369     0.000000        0.000000  proton_in_H1         2212   

     scattering_regime    helicity  \
                                     
0             coherent  conserving   
1             coherent  conserving   
2             coherent  conserving   
3             coherent  conserving   
4             coherent  conserving   
...                ...         ...   
5365              p-el    flipping   
5366              p-el    flipping   
5367              p-el    flipping   
5368              p-el    flipping   
5369              p-el    flipping   

                                     underlying_process  
                                                         
0       nu(mu) + C12 -> N5 +  C12 -> N4 + e+ + e- + C12  
1       nu(mu) + C12 -> N5 +  C12 -> N4 + e+ + e- + C12  
2       nu(mu) + C12 -> N5 +  C12 -> N4 + e+ + e- + C12  
3       nu(mu) + C12 -> N5 +  C12 -> N4 + e+ + e- + C12  
4       nu(mu) + C12 -> N5 +  C12 -> N4 + e+ + e- + C12  
...                                                 ...  
5365  nu(mu) + proton_in_H1 -> N5 +  proton_in_H1 ->...  
5366  nu(mu) + proton_in_H1 -> N5 +  proton_in_H1 ->...  
5367  nu(mu) + proton_in_H1 -> N5 +  proton_in_H1 ->...  
5368  nu(

### `GenLauncher` class
It is possible to run the generator through an instance of the `GenLauncher` class in the `DarkNews.GenLauncher` module.
We create an instance of this class, specifying the parameters of the run, then we use the `run()` method on this instance.
The created dataset can be found in the `data/` directory tree.
Alternatively, it an be accessed via the `df` attribute.

We run the same points as before, but with the `GenLauncher` class. When using the `run()` method it is possible to specify some parameters as `log`, `verbose` or `logfile` (which can be also specified during the construction of the object).

In [9]:
from DarkNews.GenLauncher import GenLauncher
gen_object = GenLauncher(mzprime=1.25, m4=0.140, neval=1000, D_or_M="dirac")
gen_object.run(log="INFO")



#########################################################
#   ______           _        _   _                     #
#   |  _  \         | |      | \ | |                    #
#   | | | |__ _ _ __| | __   |  \| | _____      _____   #
#   | | | / _  | ___| |/ /   | .   |/ _ \ \ /\ / / __|  #
#   | |/ / (_| | |  |   <    | |\  |  __/\ V  V /\__ \  #
#   |___/ \__,_|_|  |_|\_\   \_| \_/\___| \_/\_/ |___/  #
#                                                       #
#########################################################
        
Theory model used: 3+1 dirac HNL model


Generating helicity conserving upscattering events for:
	nu(mu) + C12 -> N4 +  C12 -> nu(mu) + e+ + e- + C12

decaying N4 using off-shell mediator.
Generating helicity flipping upscattering events for:
	nu(mu) + C12 -> N4 +  C12 -> nu(mu) + e+ + e- + C12

decaying N4 using off-shell mediator.
Generating helicity conserving upscattering events for:
	nu(mu) + proton_in_C12 -> N4 +  proton_in_C12 -> nu(mu) + e+ + e- + proton_

In [10]:
df_d = gen_object.df
df_d

P_projectile                     P_decay_N_parent                      \
                t    x    y         z                t         x         y   
0        0.171460  0.0  0.0  0.171460         0.171054 -0.007788  0.044171   
1        0.284386  0.0  0.0  0.284386         0.284106 -0.060913 -0.021486   
2        0.251279  0.0  0.0  0.251279         0.251134 -0.033562  0.004799   
3        0.374681  0.0  0.0  0.374681         0.374357 -0.053132 -0.055782   
4        0.236248  0.0  0.0  0.236248         0.235613 -0.062578  0.071175   
...           ...  ...  ...       ...              ...       ...       ...   
5361     2.426055  0.0  0.0  2.426055         2.263693 -0.375370 -0.370956   
5362     1.518926  0.0  0.0  1.518926         1.363585  0.379166 -0.324704   
5363     2.046125  0.0  0.0  2.046125         1.629021  0.757465  0.083491   
5364     1.828770  0.0  0.0  1.828770         1.574384 -0.607544 -0.143757   
5365     1.440907  0.0  0.0  1.440907         1.318555 -0.048587 -0.445647   

                 P_target       ... decay_displacement      w_decay_rate_0  \
             z          t    x  ...                  y    z                  
0     0.087452  11.174864 -0.0  ...                0.0  0.0   1.058787e-22   
1     0.238629  11.174864 -0.0  ...               -0.0  0.0   1.420506e-23   
2     0.205716  11.174864 -0.0  ...                0.0  0.0   7.808760e-23   
3     0.338539  11.174864 -0.0  ...               -0.0  0.0   2.542726e-23   
4     0.164109  11.174864 -0.0  ...                0.0  0.0   2.917722e-23   
...        ...        ...  ...  ...                ...  ...            ...   
5361  2.196860   0.938272 -0.0  ...               -0.0  0.0   3.387280e-22   
5362  1.261176   0.938272 -0.0  ...               -0.0  0.0   4.309246e-23   
5363  1.432964   0.938272 -0.0  ...                0.0  0.0   3.814663e-22   
5364  1.438509   0.938272 -0.0  ...               -0.0  0.0   7.540359e-23   
5365  1.232082   0.938272 -0.0  ...               -0.0  0.0   2.594279e-23   

     w_event_rate w_flux_avg_xsec        target target_pdgid  \
                                                               
0        0.000099        0.000031           C12   1000060120   
1        0.000037        0.000049           C12   1000060120   
2        0.000104        0.000071           C12   1000060120   
3        0.000104        0.000024           C12   1000060120   
4        0.000080        0.000015           C12   1000060120   
...           ...             ...           ...          ...   
5361     0.000029        0.000027  proton_in_H1         2212   
5362     0.000054        0.000080  proton_in_H1         2212   
5363     0.000039        0.000047  proton_in_H1         2212   
5364     0.000027        0.000011  proton_in_H1         2212   
5365     0.000042        0.000033  proton_in_H1         2212   

     scattering_regime    helicity  \
                                     
0             coherent  conserving   
1             coherent  conserving   
2             coherent  conserving   
3             coherent  conserving   
4             coherent  conserving   
...                ...         ...   
5361              p-el    flipping   
5362              p-el    flipping   
5363              p-el    flipping   
5364              p-el    flipping   
5365              p-el    flipping   

                                     underlying_process  
                                                         
0     nu(mu) + C12 -> N4 +  C12 -> nu(mu) + e+ + e- ...  
1     nu(mu) + C12 -> N4 +  C12 -> nu(mu) + e+ + e- ...  
2     nu(mu) + C12 -> N4 +  C12 -> nu(mu) + e+ + e- ...  
3     nu(mu) + C12 -> N4 +  C12 -> nu(mu) + e+ + e- ...  
4     nu(mu) + C12 -> N4 +  C12 -> nu(mu) + e+ + e- ...  
...                                                 ...  
5361  nu(mu) + proton_in_H1 -> N4 +  proton_in_H1 ->...  
5362  nu(mu) + proton_in_H1 -> N4 +  proton_in_H1 ->...  
5363  nu(mu) + proton_in_H1 -> N4 +  proton_in_H1 ->...  
5364  nu(

In [11]:
gen_object = GenLauncher(umu5=1e-3, ud5=35, chi=0.0031, gD=2, mzprime=1.25, m4=0.080, m5=0.140, neval=1000, D_or_M="majorana")
gen_object.run(log="INFO")



#########################################################
#   ______           _        _   _                     #
#   |  _  \         | |      | \ | |                    #
#   | | | |__ _ _ __| | __   |  \| | _____      _____   #
#   | | | / _  | ___| |/ /   | .   |/ _ \ \ /\ / / __|  #
#   | |/ / (_| | |  |   <    | |\  |  __/\ V  V /\__ \  #
#   |___/ \__,_|_|  |_|\_\   \_| \_/\___| \_/\_/ |___/  #
#                                                       #
#########################################################
        
Theory model used: 3+2 majorana HNL model


Theory model used: 3+2 majorana HNL model


Generating helicity conserving upscattering events for:
	nu(mu) + C12 -> N5 +  C12 -> N4 + e+ + e- + C12

Generating helicity conserving upscattering events for:
	nu(mu) + C12 -> N5 +  C12 -> N4 + e+ + e- + C12

decaying N5 using off-shell mediator.
decaying N5 using off-shell mediator.
Generating helicity flipping upscattering events for:
	nu(mu) + C12 -> N5 +  C12 -> N4 + e+

In [12]:
df_d = gen_object.df
df_d

P_projectile                     P_decay_N_parent                      \
                t    x    y         z                t         x         y   
0        0.600389  0.0  0.0  0.600389         0.599990  0.009691  0.090732   
1        0.298606  0.0  0.0  0.298606         0.297957  0.006894 -0.105472   
2        0.203648  0.0  0.0  0.203648         0.202908 -0.073001  0.056656   
3        0.584652  0.0  0.0  0.584652         0.584382 -0.056977 -0.047872   
4        0.581939  0.0  0.0  0.581939         0.581735  0.008025 -0.063713   
...           ...  ...  ...       ...              ...       ...       ...   
5365     1.480923  0.0  0.0  1.480923         0.626242  0.450097 -0.404832   
5366     1.284102  0.0  0.0  1.284102         0.912066  0.616349 -0.179454   
5367     1.110781  0.0  0.0  1.110781         0.643833  0.380303 -0.438646   
5368     8.089877  0.0  0.0  8.089877         6.788148  0.648404  1.266077   
5369     7.903061  0.0  0.0  7.903061         6.932040 -0.302565 -1.220962   

                 P_target       ... decay_displacement      w_decay_rate_0  \
             z          t    x  ...                  y    z                  
0     0.576248  11.174864 -0.0  ...                0.0  0.0   3.976789e-16   
1     0.240845  11.174864 -0.0  ...               -0.0  0.0   5.750206e-16   
2     0.114160  11.174864 -0.0  ...                0.0  0.0   9.177723e-16   
3     0.562463  11.174864 -0.0  ...               -0.0  0.0   5.631801e-16   
4     0.560974  11.174864 -0.0  ...               -0.0  0.0   1.015882e-15   
...        ...        ...  ...  ...                ...  ...            ...   
5365  0.078123   0.938272 -0.0  ...               -0.0  0.0   2.136635e-15   
5366  0.632593   0.938272 -0.0  ...               -0.0  0.0   7.227626e-16   
5367  0.240581   0.938272 -0.0  ...               -0.0  0.0   7.039601e-16   
5368  6.635961   0.938272 -0.0  ...                0.0  0.0   1.707376e-13   
5369  6.815518   0.938272 -0.0  ...               -0.0  0.0   1.450069e-13   

     w_event_rate w_flux_avg_xsec        target target_pdgid  \
                                                               
0       19.731114       13.008118           C12   1000060120   
1       16.920468       13.808261           C12   1000060120   
2       12.664077        9.128393           C12   1000060120   
3       21.887229       15.000501           C12   1000060120   
4       32.298143       25.976216           C12   1000060120   
...           ...             ...           ...          ...   
5365     3.870794        1.895771  proton_in_H1         2212   
5366    10.349804        5.019607  proton_in_H1         2212   
5367     9.166310        5.195656  proton_in_H1         2212   
5368     0.000000        0.000000  proton_in_H1         2212   
5369     0.000000        0.000000  proton_in_H1         2212   

     scattering_regime    helicity  \
                                     
0             coherent  conserving   
1             coherent  conserving   
2             coherent  conserving   
3             coherent  conserving   
4             coherent  conserving   
...                ...         ...   
5365              p-el    flipping   
5366              p-el    flipping   
5367              p-el    flipping   
5368              p-el    flipping   
5369              p-el    flipping   

                                     underlying_process  
                                                         
0       nu(mu) + C12 -> N5 +  C12 -> N4 + e+ + e- + C12  
1       nu(mu) + C12 -> N5 +  C12 -> N4 + e+ + e- + C12  
2       nu(mu) + C12 -> N5 +  C12 -> N4 + e+ + e- + C12  
3       nu(mu) + C12 -> N5 +  C12 -> N4 + e+ + e- + C12  
4       nu(mu) + C12 -> N5 +  C12 -> N4 + e+ + e- + C12  
...                                                 ...  
5365  nu(mu) + proton_in_H1 -> N5 +  proton_in_H1 ->...  
5366  nu(mu) + proton_in_H1 -> N5 +  proton_in_H1 ->...  
5367  nu(mu) + proton_in_H1 -> N5 +  proton_in_H1 ->...  
5368  nu(

---
# Pandas DataFrame Output

### Metadata

Before inspecting the events inside the pandas dataframe, let us check the metadata in [df.attrs](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.attrs.html). We have:

---
1. **experiment**: this is a class DarkNews.experiment.Detector, which contains all the information about the experiment for which the events are generated. It also contains a list of instances of the NuclearTarget class, which contains information about the different scattering target used.

In [19]:
print("---------------------------------------------")
print("General description of the detector attribute")
print("---------------------------------------------")
keys = list(df_d.attrs['experiment'].__dict__.keys())
print(f"Attributes of Detector class: \n", *keys)
print(f"\nAttributes of NuclearTarget class: \n", *list(df_d.attrs['experiment'].NUCLEAR_TARGETS[0].__dict__))

---------------------------------------------
General description of the detector attribute
---------------------------------------------


KeyError: 'experiment'

In [20]:
print("--------------------------------------------")
print("How much of each target inside the detector?")
print("--------------------------------------------")
print("\nExample of how to look up nuclear targets:")
fid_mass = df_d.attrs['experiment'].FIDUCIAL_MASS
for target, fid_mass_fraction in zip(df_d.attrs['experiment'].NUCLEAR_TARGETS, df_d.attrs['experiment'].FIDUCIAL_MASS_FRACTION_PER_TARGET):
    print("{} tonnes of {}".format(round(fid_mass_fraction*fid_mass,2), target.NAME))

--------------------------------------------
How much of each target inside the detector?
--------------------------------------------

Example of how to look up nuclear targets:


KeyError: 'experiment'

---
2. **bsm_model**: this attribute contains DarkNews.model.Model, which stores all the new physics parameters used in the generation. This includes both low-level information, such as the couplings, say $g_D$, as well as high-level ones, such as the couplings of the $Z^\prime$ to electrons, $d_e^V$ and $d_e^A$.

In [21]:
print("Some low-level parameters:\n", *list(df_d.attrs['bsm_model'].__dict__.keys())[:19])
print("\nSome high-level parameters:\n", *list(df_d.attrs['bsm_model'].__dict__.keys())[-14:])

KeyError: 'bsm_model'

---
### Description of the event structure
Now we look at the actual events. The DataFrame is a MultiIndex df. Each event contains the components for all the 4-momenta of the particles involved:

$$\nu _\text{P_projectile} \,+\, \text{Hadronic target} _\text{P_target} \to N _\text{P_decay_N_parent} \,+\, \text{Hadronic recoil} _\text{P_recoil}$$

$$N _\text{P_decay_N_parent} \to N^\prime _\text{P_decay_N_daughter} \,+\, \ell^+ _\text{P_decay_ell_plus} \,+\, \ell^-_\text{P_decay_ell_minus}$$

In [22]:
df_d.dtypes

P_projectile        t    float64
                    x    float64
                    y    float64
                    z    float64
P_decay_N_parent    t    float64
                    x    float64
                    y    float64
                    z    float64
P_target            t    float64
                    x    float64
                    y    float64
                    z    float64
P_recoil            t    float64
                    x    float64
                    y    float64
                    z    float64
P_decay_ell_minus   t    float64
                    x    float64
                    y    float64
                    z    float64
P_decay_ell_plus    t    float64
                    x    float64
                    y    float64
                    z    float64
P_decay_N_daughter  t    float64
                    x    float64
                    y    float64
                    z    float64
decay_displacement  t    float64
                    x    float64
          

##

* w_decay_rate_0           float64 -- weight of the decay rate (sum(w) = Gamma_N)
* I_decay_rate_0           float64 -- total rate Gamma_N


* w_event_rate             float64 -- weight for the event rate (sum(w) = event rate)
* I_event_rate             float64 -- total event rate


* w_flux_avg_xsec          float64 -- weight of the flux averaged cross section (sum(w) = int(sigma*flux)* exposure)
* I_flux_avg_xsec          float64 -- int(sigma*flux)* exposure


* target                    object -- target object -- it will typically be a nucleus
* target_pdgid               int64 -- pdgID of the target 


* scattering_regime         object -- regime can be coherent or p-elastic
* helicity                  object -- helicity process, can be flipping or conserving. flipping is suppressed
* underlying_process        object -- string of the underlying process, e.g, "nu(mu) + proton_in_C12 -> N4 +  proton_in_C12 -> nu(mu) + e+ + e- + proton_in_C12"